In [22]:
%load_ext autoreload
%autoreload 2

import docker
import logging

from pymongo import MongoClient
from jtlutil.docker.manager import * 
from jtlutil.docker.db import * 
from jtlutil.config import get_config

config = get_config('./config/config.env')

client = MongoClient(config.MONGO_PUBLIC)

dcsr = DockerContainerStatsRepository(client)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from pydantic import Field
from faker import Faker

fake = Faker()

def create_fake_docker_container_stats() -> DockerContainerStats:
    return DockerContainerStats(
        id=fake.uuid4(),
        state=fake.word(),
        name=fake.word(),
        memory_usage=fake.random_number(digits=6),
        hostname=fake.hostname(),
        node=fake.word(),
        created=fake.date_time().isoformat()
    )


In [18]:
# Generate and insert 10 fake Docker container stats into the repository
for _ in range(10):
    fake_stat = create_fake_docker_container_stats()
    dcsr.add(fake_stat)

print("Inserted 10 fake Docker container stats into the repository.")

N = len(dcsr.all)
print("N=", N)

for record in  dcsr.all:
    new_fake_stat = create_fake_docker_container_stats()
    new_fake_stat.id = record.id
    dcsr.add(new_fake_stat)

# Check that another call to dcsr.all only has 10 records
assert len(dcsr.all) == N, "The number of records in the repository is not 10"

for record in  dcsr.all:
    new_fake_stat = create_fake_docker_container_stats()
    new_fake_stat.name = record.name
    dcsr.add(new_fake_stat)

# Check that another call to dcsr.all only has 10 records
assert len(dcsr.all) == N, "The number of records in the repository is not 10"

for record in  dcsr.all:
    new_fake_stat = create_fake_docker_container_stats()
    new_fake_stat.name = record.name
    new_fake_stat.id = record.id
    dcsr.add(new_fake_stat)

# Check that another call to dcsr.all only has 10 records
assert len(dcsr.all) == N, "The number of records in the repository is not 10"



Inserted 10 fake Docker container stats into the repository.
N= 20


In [19]:
# Delete all records from the repository
for record in dcsr.all:
    dcsr.delete(record.id)

# Verify that the repository is empty
assert len(dcsr.all) == 0, "The repository is not empty"
print("All records have been deleted from the repository.")

All records have been deleted from the repository.


In [ ]:
swarm_client = docker.DockerClient(base_url=config.SSH_URI_SWARM )
sm = ServicesManager(swarm_client, hostname_f=hostname_f)